In [7]:
def view_csv_head_native(file_path: str, n_rows: int = 20, encoding: str = "utf-8"):
    try:
        with open(file_path, 'r', encoding=encoding) as f:
            # 读取表头
            header = f.readline().strip()
            print(f"表头：{header}")
            print("-" * 80)
            
            # 读取前n行内容
            for i in range(n_rows):
                line = f.readline()
                if not line:  # 文件不足100行时终止
                    break
                print(f"第{i+1}行：{line.strip()}")
                
    except FileNotFoundError:
        print(f"文件不存在：{file_path}")
    except UnicodeDecodeError:
        print(f"编码错误，尝试GBK编码...")
        view_csv_head_native(file_path, n_rows, encoding="gbk")

# 调用
view_csv_head_native(r"C:\Users\31392\Desktop\writebyhand\wmt_data\wmt_zh_en_training_corpus.csv", n_rows=20)

表头：0,1
--------------------------------------------------------------------------------
第1行：表演 的 明星 是 X 女孩 团队 — — 由 一对 具有 天才 技艺 的 艳舞 女孩 们 组成 ， 其中 有些 人 受过 专业 的 训练 。,"the show stars the X Girls - a troupe of talented topless dancers , some of whom are classically trained ."
第2行：表演 的 压轴戏 是 闹剧 版 《 天鹅湖 》 ， 男女 小 人们 身着 粉红色 的 芭蕾舞 裙 扮演 小天鹅 。,the centerpiece of the show is a farcical rendition of Swan Lake in which male and female performers dance in pink tutus and imitate swans .
第3行：表演 和 后期制作 之间 的 屏障 被 清除 了 ， 这 对 演员 来说 一样 大有裨益 。,the removal of the barrier between performance and post @-@ production was just as helpful for the actors .
第4行：（ 表演 或 背诵 时 ） 通过 暗示 下面 忘记 或 记地 不准 的 东西 来 帮助 某人 。,assist ( somebody acting or reciting ) by suggesting the next words of something forgotten or imperfectly learned .
第5行：表演 基本上 很 精彩 - - 我 只 对 她 的 技巧 稍 有 意见 。,basically it was a fine performance I have only minor quibbles to make about her technique .
第6行：表演 结束 后 ， 我们 看到 一对对 车灯 沿主路 一路 排回 镇上 ， 然后 散开 来 各回 各家 。,"af

In [8]:
import os

def count_csv_total_lines(file_path: str, encoding: str = "utf-8") -> int:
    """
    统计CSV文件总行数（高效版，逐行读取，低内存占用）
    :param file_path: CSV文件路径
    :param encoding: 文件编码，默认UTF-8
    :return: 总行数（包含表头）
    """
    # 第一步：检查文件是否存在
    if not os.path.exists(file_path):
        print(f"❌ 错误：文件不存在 → {file_path}")
        return 0
    
    # 第二步：逐行统计行数
    total_lines = 0
    try:
        with open(file_path, 'r', encoding=encoding) as f:
            # 可选：显示文件大小，让你有预期
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # 转换为MB
            print(f"📄 文件大小：{file_size:.2f} MB")
            print("🔢 正在统计行数...（超大文件可能需要几秒）")
            
            # 逐行读取，统计行数
            for _ in f:
                total_lines += 1
                
        print(f"✅ 统计完成！")
        print(f"📊 CSV文件总行数（含表头）：{total_lines}")
        print(f"📊 数据行数（不含表头）：{total_lines - 1 if total_lines > 0 else 0}")
        return total_lines
    
    except UnicodeDecodeError:
        print(f"❌ 错误：{encoding}编码解析失败，尝试GBK编码重新统计...")
        # 自动重试GBK编码（中文Windows文件常见）
        return count_csv_total_lines(file_path, encoding="gbk")
    except Exception as e:
        print(f"❌ 未知错误：{str(e)}")
        return 0

# ------------------- 核心调用 -------------------
if __name__ == "__main__":
    # 替换为你的CSV文件实际路径
    csv_file_path = r"C:\Users\31392\Desktop\writebyhand\wmt_data\wmt_zh_en_training_corpus.csv"
    
    # 统计总行数
    count_csv_total_lines(csv_file_path)

📄 文件大小：6064.67 MB
🔢 正在统计行数...（超大文件可能需要几秒）
✅ 统计完成！
📊 CSV文件总行数（含表头）：24752393
📊 数据行数（不含表头）：24752392


In [1]:
import html
from pathlib import Path
import numpy as np

def tokenize_space(text: str):
    return [t for t in text.strip().split() if t]

def load_pairs_from_csv_first_comma(path, max_rows=None):
    path = Path(path)
    pairs = []
    with path.open("r", encoding="utf-8") as f:
        for line_idx, line in enumerate(f):
            line = line.strip()
            if not line:
                continue

            # 跳过表头
            if line_idx == 0:
                head = line.replace('"', "").replace(" ", "")
                if head == "0,1":
                    continue

            k = line.find(",")
            if k == -1:
                continue

            src = line[:k].strip()
            tgt = line[k+1:].strip()

            # 去两侧引号（若有）
            if len(src) >= 2 and src[0] == '"' and src[-1] == '"':
                src = src[1:-1].strip()
            if len(tgt) >= 2 and tgt[0] == '"' and tgt[-1] == '"':
                tgt = tgt[1:-1].strip()

            src = html.unescape(src)
            tgt = html.unescape(tgt)

            if src and tgt:
                pairs.append((src, tgt))

            if max_rows is not None and len(pairs) >= max_rows:
                break
    return pairs

def analyze_max_lengths(pairs, show_chars=200):
    src_lens = []
    tgt_lens = []
    max_src = (-1, None)  # (len, text)
    max_tgt = (-1, None)

    for src, tgt in pairs:
        s_len = len(tokenize_space(src))
        t_len = len(tokenize_space(tgt))

        src_lens.append(s_len)
        tgt_lens.append(t_len)

        if s_len > max_src[0]:
            max_src = (s_len, src)
        if t_len > max_tgt[0]:
            max_tgt = (t_len, tgt)

    src_arr = np.array(src_lens)
    tgt_arr = np.array(tgt_lens)

    def stats(arr):
        return {
            "count": int(arr.size),
            "p50": float(np.percentile(arr, 50)),
            "p90": float(np.percentile(arr, 90)),
            "p95": float(np.percentile(arr, 95)),
            "p99": float(np.percentile(arr, 99)),
            "max": int(arr.max()),
            "mean": float(arr.mean()),
        }

    print("=== SRC length stats (space-token count) ===")
    print(stats(src_arr))
    print("\n=== TGT length stats (space-token count) ===")
    print(stats(tgt_arr))

    print("\n=== Longest SRC ===")
    print("len =", max_src[0])
    print(max_src[1][:show_chars] + ("..." if len(max_src[1]) > show_chars else ""))

    print("\n=== Longest TGT ===")
    print("len =", max_tgt[0])
    print(max_tgt[1][:show_chars] + ("..." if len(max_tgt[1]) > show_chars else ""))

    return {
        "src_lens": src_arr,
        "tgt_lens": tgt_arr,
        "max_src_len": max_src[0],
        "max_tgt_len": max_tgt[0],
        "max_src_text": max_src[1],
        "max_tgt_text": max_tgt[1],
    }

# ===== 用法 1：如果你还没读全量 pairs =====
csv_path = "./wmt_data/wmt_zh_en_training_corpus.csv"  # 改成你的实际路径
pairs_all = load_pairs_from_csv_first_comma(csv_path)

result = analyze_max_lengths(pairs_all, show_chars=240)


=== SRC length stats (space-token count) ===
{'count': 24752356, 'p50': 17.0, 'p90': 39.0, 'p95': 47.0, 'p99': 61.0, 'max': 80, 'mean': 19.793406130713375}

=== TGT length stats (space-token count) ===
{'count': 24752356, 'p50': 21.0, 'p90': 46.0, 'p95': 56.0, 'p99': 74.0, 'max': 155, 'mean': 23.971527276029803}

=== Longest SRC ===
len = 80
我 觉得 我 依从 了 ， 可以 免去 另 一场 乱子 ； 我 也 认为 ， 这 也许 可以 在 凯瑟琳 的 心病 上 创造 一个 有利 的 转机 ： 后来 我 又 记起 埃德加 先生 严厉 责骂 我 搬弄是非 ； 我 反复 肯定 说 那次 背信 告密 的 事 ， 如果 该受 这样 粗暴 的 名称 的话 ， 也 该是 最后 一次 了 ， 我 借 这个 肯定 来 消除 我 对于 这事 所 感到 的 一切 不安 。

=== Longest TGT ===
len = 155
委员会 决定 设立 一个 由 Awosika 先生 担任 主席 的 不限 成员 名额 工作组 ( 成员 包括 Albuquerque 先生 、 Beltagy 先生 、 Betah 先生 、 Brekke 先生 、 Carrera 先生 、 Chan Chim Yuk 先生 、 Francis 先生 、 Hamuro 先生 、 Hinz 先生 、 Jaafar 先生 、 吕文正 先生 、 Park 先生 和 Srinivasan 先生 ) , 负责 在 闭会 期间 草拟 关于...
